In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm.notebook.tqdm as tqdm
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

## Se importan los dataframes 

In [2]:
# load all metadata
#hau nire diskan daukat SAD karpetan 2020-2021

video_games = []#
insurances = []   
#dfMergedfMeta= pd.read_csv('Cell_Phones/SamsungAppleXiaomiReviews_prueba.csv')
video_games= pd.read_csv('VG-Reviews5AndMetaElecNintSonyMic_v2.csv')
insurances = pd.read_csv('HRBlockIntuitReviewsTrainDev_vLast7.csv', low_memory=False)


FileNotFoundError: [Errno 2] No such file or directory: 'VG-Reviews5AndMetaElecNintSonyMic_v2.csv'

In [ ]:
video_games.keys()

In [ ]:
video_games['brand'].value_counts()

In [ ]:
insurances.keys()

In [ ]:
insurances['brand'].value_counts()

## Se obtienen en dataframes aparte las distintas marcas

In [ ]:
dfNintendo = video_games[video_games['brand'].str.contains('Nintendo',na=False)]

In [ ]:
dfSony = video_games[video_games['brand'].str.contains('Sony|PlayStation|Electr',na=False)]

In [ ]:
dfMicrosoft = video_games[video_games['brand'].str.contains('[M|m]icrosoft',na=False)]

In [ ]:
dfIntuit = insurances[insurances['brand'].str.contains('Intuit')]

In [ ]:
dfHR = insurances[~insurances['brand'].str.contains('Intuit')]

## Se separan las reviews negativas y positivas de cada dataframe

In [ ]:
dfNintendoPositives=dfNintendo[dfNintendo['overall']>3]
dfNintendoNegatives=dfNintendo[dfNintendo['overall']<=3]

In [ ]:
dfSonyPositives=dfSony[dfSony['overall']>3]
dfSonyNegatives=dfSony[dfSony['overall']<=3]

In [ ]:
dfMicrosoftPositives=dfMicrosoft[dfMicrosoft['overall']>3]
dfMicrosoftNegatives=dfMicrosoft[dfMicrosoft['overall']<=3]

In [ ]:
dfIntuitPositives = dfIntuit[dfIntuit['overall']>3]
dfIntuitNegatives = dfIntuit[dfIntuit['overall']<=3]

In [ ]:
dfHRPositives = dfHR[dfHR['overall']>3]
dfHRNegatives = dfHR[dfHR['overall']<=3]

## Se obtienen las text reviews

In [ ]:
nintendoPositivesDocuments = dfNintendoPositives[dfNintendoPositives['reviewText'].notna()]['reviewText'].tolist()
nintendoNegativesDocuments = dfNintendoNegatives[dfNintendoNegatives['reviewText'].notna()]['reviewText'].tolist()

In [ ]:
sonyPositivesDocuments = dfSonyPositives[dfSonyPositives['reviewText'].notna()]['reviewText'].tolist()
sonyNegativesDocuments = dfSonyNegatives[dfSonyNegatives['reviewText'].notna()]['reviewText'].tolist()

In [ ]:
microsoftPositivesDocuments = dfMicrosoftPositives[dfMicrosoftPositives['reviewText'].notna()]['reviewText'].tolist()
microsoftNegativesDocuments = dfMicrosoftNegatives[dfMicrosoftNegatives['reviewText'].notna()]['reviewText'].tolist()

In [ ]:
intuitPositivesDocuments = dfIntuitPositives[dfIntuitPositives['reviewText'].notna()]['reviewText'].tolist()
intuitNegativesDocuments = dfIntuitNegatives[dfIntuitNegatives['reviewText'].notna()]['reviewText'].tolist()

In [ ]:
HRPositivesDocuments = dfHRPositives[dfHRPositives['reviewText'].notna()]['reviewText'].tolist()
HRNegativesDocuments = dfHRNegatives[dfHRNegatives['reviewText'].notna()]['reviewText'].tolist()

## LDA

In [ ]:
def display_topics(H, W, feature_names, documents,output_name, no_top_words = 30, no_top_documents = 10):
    f = open(output_name + ".txt", "a")
    for topic_idx, topic in enumerate(H):
        s = "Topic: " + str(topic_idx) + " -->"
        print(s)
        f.write(s)
        s = ''.join([' ' +feature_names[i] + ' ' + str(round(topic[i], 5)) 
                for i in topic.argsort()[:-no_top_words - 1:-1]])
        print(s)
        f.write(s)
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        docProbArray=np.argsort(W[:,topic_idx])
        print(docProbArray)
    
        #for doc_index in top_doc_indices:
        #    print(documents[doc_index])
        f.write("\n")
    f.close()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### LDANintPos_31_1.9

In [ ]:
no_topics = 31

alpha = 1.9
beta = 1.9

documents = nintendoPositivesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDANintPos_31_1.9')
#no_top_words = 30 
#no_top_documents = 10

### LDANintNeg_26_1.9

In [ ]:
no_topics = 26

alpha = 1.9
beta = 1.9

documents = nintendoNegativesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDANintNeg_26_1.9')

### LDASonyPos_34_1.9

In [ ]:
no_topics = 34

alpha = 1.9
beta = 1.9

documents = sonyPositivesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDASonyPos_34_1.9')

## GENSIM

In [ ]:
import logging
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from pprint import pprint
nltk.download('wordnet')
nltk.download('omw-1.4')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
def preproceso(docs):
    # Tokenize 
    tokenizer = RegexpTokenizer(r'\w+')
    for idx in range(len(docs)):
        docs[idx] = docs[idx].lower()  # Convert to lowercase.
        docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

    # Remove numbers, but not words that contain numbers.
    docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

    # Remove words that are only one character.
    docs = [[token for token in doc if len(token) > 1] for doc in docs]

    # Remove stopwords
    stopwords = gensim.parsing.preprocessing.STOPWORDS
    docs = [[token for token in doc if not(token in stopwords)] for doc in docs]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

    # Bag-of-words
    dictionary = Dictionary(docs)

    corpus = [dictionary.doc2bow(doc) for doc in docs]
    print('Number of unique tokens: %d' % len(dictionary))
    print('Number of documents: %d' % len(corpus))

    return corpus , dictionary

In [ ]:
def gen_model(num_topics,corpus, dictionary):
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = None  # Don't evaluate model perplexity, takes too much time.

    # Make an index to word dictionary.
    temp = dictionary[0]  # This is only to "load" the dictionary.
    id2word = dictionary.id2token

    model = LdaModel(
        corpus=corpus,
        id2word=id2word,
        chunksize=chunksize,
        alpha='auto',
        eta='auto',
        iterations=iterations,
        num_topics=num_topics,
        passes=passes,
        eval_every=eval_every
    )

    return model

### GensimSonyPos

In [ ]:
corpus,dictionary = preproceso(sonyPositivesDocuments)

In [ ]:
f = open('GensimSonyPos[10-100]' + ".txt", "a")
for i in range(10,101):
    print("Generando modelo: " + str(i))
    model_act=gen_model(i,corpus,dictionary)
    top_topics = model_act.top_topics(corpus)
    f.write('Modelo con %i clusters -->' % i)
    for frec, word in top_topics[0][0]:
        f.write(" " + word + " " + str(frec) + " |")
    f.write("\n\n")
f.close()

### GensimSonyNeg

In [ ]:
corpus,dictionary = preproceso(sonyNegativesDocuments)

In [ ]:
f = open('GensimSonyNeg[10-100]' + ".txt", "a")
for i in range(10,101):
    print("Generando modelo: " + str(i))
    model_act=gen_model(i,corpus,dictionary)
    top_topics = model_act.top_topics(corpus)
    f.write('Modelo con %i clusters -->' % i)
    for frec, word in top_topics[0][0]:
        f.write(" " + word + " " + str(frec) + " |")
    f.write("\n\n")
f.close()